# <h1><center> El Problema de Transporte y Asignación </center></h1>

### Seminario: Introducción a los Modelos de Optimización - ESE-IPN

A continuación describimos el modelo de optimización del problema de transporte y asignación.

##### Conjuntos

<ol>
  <li>$O$, conjunto de puntos de orígen $i$.</li>
  <li>$D$, conjunto de puntos de destino $j$.</li> 
</ol>

##### Parámetros

<ol>
  <li>$p_i \geq 0$, cantidad de producción de la unidad de orígen $i\in O$,</li>
  <li>$d_j \geq 0$, cantidad de la demanda de la unidad de destino $j\in D$,</li>
  <li>$c_{i,j} \geq 0$, costo de transporte por unidad de producto del orígen $i$ al destino $j$,</li>
</ol>


##### Variables 

<ol>
  <li>$x_{i,j} \geq 0$, cantidad de producto que se transportará del origen $i\in O$ al destino $j \in D$.</li>
</ol>

##### Función objetivo 

$$
\min \sum_{i \in O, j \in D} c_{i,j}x_{i,j} \tag{1}
$$

##### Restricciones 
$$
\sum_{i \in O}p_i = \sum_{j \in D}d_j,  \tag{2}
$$
$$
\sum_{j \in D} x_{i,j}=p_i ~~~~ \text{para cada} ~~~~  i\in O,  \tag{3}
$$
$$
\sum_{i \in O} x_{i,j}=d_j ~~~~ \text{para cada} ~~~~  j\in D.  \tag{4}
$$

Primero importamos los módulos de Python que usaremos para resolver nuestro problema. 

In [6]:
#%%Se importan los módulos necesarios ------------------------------------------
import pandas as pd                                                            #para manipular tablas
#%load_ext ampl
from pulp import LpProblem, LpVariable, LpMinimize,lpSum, value                #para el modelo de optimización 
#------------------------------------------------------------------------------

Luego leemos los parámetros del modelo. Para este ejemplo, nuestros datos estan en un archivo Excel.  

In [7]:
file="C:/Users/A1644/OneDrive - Centro Nacional de Control de Energia (CENACE)/ESE-IPN- II-2017/Optimización/Python/BD_transport.xlsx"

x=pd.ExcelFile(file)                                                           #lee el archivo excel

DF_O=x.parse('ORIG',index_col="ORIG")                                          #lee las tablas del archivo excel
DF_D=x.parse('DEST',index_col="DEST")
DF_CT=x.parse('COST',index_col="COST") 

print("-"*50)
print(DF_O)
print("-"*50)
print(DF_D)
print("-"*50)
print(DF_CT)
print("-"*50)

--------------------------------------------------
      prod
ORIG      
GARY  1400
CLEV  2600
PITT  2900
--------------------------------------------------
       dem
DEST      
FRA    900
DET   1200
LAN    600
WIN    400
STL   1700
FRE   1100
LAF   1000
--------------------------------------------------
      FRA  DET  LAN  WIN  STL  FRE  LAF
COST                                   
GARY   39   14   11   14   16   82    8
CLEV   27    9   12    9   26   95   17
PITT   24   14   17   13   28   99   20
--------------------------------------------------


Ahora escribimos el modelo.

In [19]:
#%% Se define el problema------------------------------------------------------
Probt=LpProblem("Total_Cost",LpMinimize)

#se declaran las variables del modelo de optimización -------------------------
X=LpVariable.dicts("transporte", [(x, y) for x in set(DF_O.index) for y in set(DF_D.index)], lowBound=0, cat="Continuous")  

#Se define la función objetivo ------------------------------------------------ 
Probt+=lpSum(DF_CT.loc[i,j]*X[i,j] for i in set(DF_O.index) for j in set(DF_D.index))

#Se agregan las restricciones ------------------------------------------------- 
#for i in DF_F.index:
#    Prob+= X[i] >= DF_F.loc[i,"f_min"]
#    Prob+= X[i] <= DF_F.loc[i,"f_max"]

#Se agregan las restricciones ------------------------------------------------- 
Probt+= lpSum(DF_O.loc[i,"prod"] for i in set(DF_O.index))== lpSum(DF_D.loc[j,"dem"] for j in set(DF_D.index))
#la anterior restricción solo es de verificación; no involucra ninguna variable

for i in set(DF_O.index):
    Probt+= lpSum(X[i,j] for j in set(DF_D.index))== DF_O.loc[i,"prod"]

for j in set(DF_D.index):
    Probt+= lpSum(X[i,j] for i in set(DF_O.index))== DF_D.loc[j,"dem"]

#Se elige el solver------------------------------------------------------------ 
Probt.solve()    
    

1

NOTA.  Pulp no acepta las siguentes formas de restricción: 

A continuación mostramos la solución del problema de optimización.

In [18]:
for v in Probt.variables():
    print(v.name,"=",v.varValue)                                               #valor de las variables         

print("-"*50)
print("Valor Obj.","=", value(Probt.objective)) 

transporte_('CLEV',_'DET') = 1200.0
transporte_('CLEV',_'FRA') = 0.0
transporte_('CLEV',_'FRE') = 0.0
transporte_('CLEV',_'LAF') = 400.0
transporte_('CLEV',_'LAN') = 600.0
transporte_('CLEV',_'STL') = 0.0
transporte_('CLEV',_'WIN') = 400.0
transporte_('GARY',_'DET') = 0.0
transporte_('GARY',_'FRA') = 0.0
transporte_('GARY',_'FRE') = 1100.0
transporte_('GARY',_'LAF') = 300.0
transporte_('GARY',_'LAN') = 0.0
transporte_('GARY',_'STL') = 0.0
transporte_('GARY',_'WIN') = 0.0
transporte_('PITT',_'DET') = 0.0
transporte_('PITT',_'FRA') = 900.0
transporte_('PITT',_'FRE') = 0.0
transporte_('PITT',_'LAF') = 300.0
transporte_('PITT',_'LAN') = 0.0
transporte_('PITT',_'STL') = 1700.0
transporte_('PITT',_'WIN') = 0.0
--------------------------------------------------
Valor Obj. = 196200.0


La solución anterior coencide con el que se muestra en la Página 35 del manual de AMPL.

In [1]:
%%ampl


ERROR:root:Cell magic `%%ampl` not found.
